In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Data Loading and Preprocessing**

In [ ]:
import os
import pandas as pd
import numpy as np
img_dir = '/content/drive/MyDrive/Colab Notebooks/train/'
train_data_source = "/content/drive/MyDrive/Colab Notebooks/train.csv"
test_dir = '/content/drive/MyDrive/Colab Notebooks/test/'
test_data_source = "/content/drive/MyDrive/Colab Notebooks/test.csv"

img_names =  sorted(os.listdir(img_dir), key=lambda x: int(x.split('.')[0]))
test_image_names =  sorted(os.listdir(test_dir), key=lambda x: int(x.split('.')[0]))
img_data_df = pd.read_csv(train_data_source)
img_data_df['image_name'] = img_names
test_data_df = pd.read_csv(test_data_source)
test_data_df['image_name'] = test_image_names

num_classes = 6
img_data_df['actual_stable_height'] = img_data_df['stable_height'] - 1
img_data_df['actual_stable_height'] = img_data_df['actual_stable_height'].astype(str)

In [18]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input  # Import ResNet and preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train, validation = train_test_split(img_data_df, test_size=0.2, random_state=42)

data_generator = ImageDataGenerator(
    rescale=1./128,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

train_data = data_generator.flow_from_dataframe(
    dataframe=train,
    directory=img_dir,
    x_col='image_name',
    y_col='actual_stable_height',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

validation_data = data_generator.flow_from_dataframe(
    dataframe=validation,
    directory=img_dir,
    x_col='image_name',
    y_col='actual_stable_height',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

data_generator = ImageDataGenerator(
    rescale=1./128,
    preprocessing_function=preprocess_input
)

data_test = data_generator.flow_from_dataframe(
    dataframe=test_data_df,
    directory=test_dir,
    x_col='image_name',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

images, labels = next(train_data)
train_batch_size = train_data.batch_size
train_batch_idx = train_data.batch_index - 1
start_idx =  train_batch_idx * train_batch_size

for i in range(len(images)):
    label = labels[i]
    image = images[i]
    image_name = train_data.filenames[start_idx + i]
    image = image * 128

images = next(data_test)
batch_size_test = data_test.batch_size
test_batch_idx = data_test.batch_index - 1
start_idx = test_batch_idx * batch_size_test

for i in range(len(images)):
    image = images[i]
    image_name = data_test.filenames[start_idx + i]
    image = image * 128

# Build Models

**Train and Predict**

In [12]:
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionResNetV2
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Load base models without the top and set layers to trainable
base_model1 = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))
base_model2 = VGG19(weights=None, include_top=False, input_shape=(224, 224, 3))
base_model3 = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))
base_model4 = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 3))
base_model5 = InceptionResNetV2(weights=None, include_top=False, input_shape=(224, 224, 3))

for layer in base_model1.layers:
    layer.trainable = True
for layer in base_model2.layers:
    layer.trainable = True
for layer in base_model3.layers:
    layer.trainable = True
for layer in base_model4.layers:
    layer.trainable = True
for layer in base_model5.layers:
    layer.trainable = True

In [13]:
# Add custom layers on top of the base model
model1 = tf.keras.Sequential([
    base_model1,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [14]:
model1.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]

In [15]:
history = model1.fit(train_data,
                    epochs=200,
                    validation_data=validation_data,
                    callbacks=callbacks)

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


192/192 ━━━━━━━━━━━━━━━━━━━━ 4959s 25s/step - accuracy: 0.2448 - loss: 1.7264 - val_accuracy: 0.2415 - val_loss: 1.7059
Epoch 2/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 92s 466ms/step - accuracy: 0.2452 - loss: 1.6919 - val_accuracy: 0.2415 - val_loss: 1.7059
Epoch 3/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 93s 469ms/step - accuracy: 0.2498 - loss: 1.6720 - val_accuracy: 0.2415 - val_loss: 1.7038
Epoch 4/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 460ms/step - accuracy: 0.2561 - loss: 1.6706 - val_accuracy: 0.2539 - val_loss: 1.6680
Epoch 5/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 461ms/step - accuracy: 0.2474 - loss: 1.6376 - val_accuracy: 0.2526 - val_loss: 1.6051
Epoch 6/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 92s 464ms/step - accuracy: 0.2510 - loss: 1.6036 - val_accuracy: 0.2376 - val_loss: 1.5884
Epoch 7/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 461ms/step - accuracy: 0.2520 - loss: 1.5898 - val_accuracy: 0.2533 - val_loss: 1.6073
Epoch 8/200
192/192 ━━━━━━━━━━━━━━━━━━━━ 92s 463ms/step - accuracy: 0.2454 - loss: 1.572

In [16]:
predictions = model1.predict(data_test)

60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 99ms/step


In [17]:
stable_height = np.argmax(predictions, axis=1) + 1

output = pd.DataFrame({
    'id': test_data_df['id'],
    'stable_height': stable_height
})

output.to_csv('/content/drive/MyDrive/Colab Notebooks/predictions.csv', index=False)

In [ ]:
history = model1.fit(train_data,
                    epochs=100,
                    validation_data=validation_data,
                    callbacks=callbacks)

Epoch 1/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 462ms/step - accuracy: 0.2781 - loss: 1.5983 - val_accuracy: 0.3372 - val_loss: 1.5265
Epoch 2/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 457ms/step - accuracy: 0.3569 - loss: 1.4917 - val_accuracy: 0.4049 - val_loss: 1.4350
Epoch 3/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 461ms/step - accuracy: 0.3899 - loss: 1.4270 - val_accuracy: 0.4030 - val_loss: 1.3792
Epoch 4/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 459ms/step - accuracy: 0.4271 - loss: 1.3671 - val_accuracy: 0.4629 - val_loss: 1.3076
Epoch 5/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 460ms/step - accuracy: 0.4439 - loss: 1.3301 - val_accuracy: 0.4655 - val_loss: 1.2929
Epoch 6/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 457ms/step - accuracy: 0.4638 - loss: 1.2897 - val_accuracy: 0.4674 - val_loss: 1.2922
Epoch 7/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 458ms/step - accuracy: 0.4794 - loss: 1.2500 - val_accuracy: 0.5091 - val_loss: 1.2174
Epoch 8/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 456ms/step - accuracy: 0.4888 -

In [ ]:
model1 = tf.keras.Sequential([
    base_model1,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model1.fit(train_data,
                    epochs=100,
                    validation_data=validation_data,
                    callbacks=callbacks)

Epoch 1/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 461ms/step - accuracy: 0.2499 - loss: 1.6705 - val_accuracy: 0.2467 - val_loss: 1.6565
Epoch 2/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 461ms/step - accuracy: 0.2904 - loss: 1.6009 - val_accuracy: 0.3418 - val_loss: 1.5411
Epoch 3/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 458ms/step - accuracy: 0.3628 - loss: 1.4963 - val_accuracy: 0.4049 - val_loss: 1.4482
Epoch 4/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 460ms/step - accuracy: 0.4079 - loss: 1.4152 - val_accuracy: 0.4076 - val_loss: 1.3857
Epoch 5/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 461ms/step - accuracy: 0.4459 - loss: 1.3369 - val_accuracy: 0.4486 - val_loss: 1.3350
Epoch 6/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 457ms/step - accuracy: 0.4674 - loss: 1.2669 - val_accuracy: 0.5143 - val_loss: 1.2062
Epoch 7/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 456ms/step - accuracy: 0.5170 - loss: 1.2076 - val_accuracy: 0.4967 - val_loss: 1.2093
Epoch 8/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 460ms/step - accuracy: 0.5462 -

**VGG 19**

In [ ]:
model2 =  tf.keras.Sequential([
    base_model2,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='softmax')
])

model2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model2.fit(train_data,
                    epochs=150,
                    validation_data=validation_data,
                    callbacks=callbacks)

Epoch 1/150
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 459ms/step - accuracy: 0.2369 - loss: 1.6861 - val_accuracy: 0.2415 - val_loss: 1.7036
Epoch 2/150
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 459ms/step - accuracy: 0.2520 - loss: 1.6776 - val_accuracy: 0.2415 - val_loss: 1.7045
Epoch 3/150
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 454ms/step - accuracy: 0.2593 - loss: 1.6650 - val_accuracy: 0.2415 - val_loss: 1.7025
Epoch 4/150
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 455ms/step - accuracy: 0.2579 - loss: 1.6785 - val_accuracy: 0.2402 - val_loss: 1.6837
Epoch 5/150
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 457ms/step - accuracy: 0.2461 - loss: 1.6351 - val_accuracy: 0.2422 - val_loss: 1.6372
Epoch 6/150
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 455ms/step - accuracy: 0.2541 - loss: 1.5987 - val_accuracy: 0.2422 - val_loss: 1.5847
Epoch 7/150
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 453ms/step - accuracy: 0.2567 - loss: 1.5818 - val_accuracy: 0.2409 - val_loss: 1.5853
Epoch 8/150
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 454ms/step - accuracy: 0.2587 -

### **Ensemble Method**

In [ ]:
# Inputs
input_tensor = layers.Input(shape=(224, 224, 3))

# Pass the same input through both models
x1 = base_model1(input_tensor)
x2 = base_model4(input_tensor)

# Add pooling layers
x1 = layers.GlobalAveragePooling2D()(x1)
x2 = layers.GlobalAveragePooling2D()(x2)

# Concatenate the outputs
concatenated = layers.Concatenate()([x1, x2])

# Add custom layers on top
x = layers.BatchNormalization()(concatenated)
x = layers.Dense(200, activation='relu')(x)
x = layers.Dropout(0.2)(x)
final_output = layers.Dense(6, activation='softmax')(x)

# Create the final model
ensemble_model = Model(inputs=input_tensor, outputs=final_output)

# Compile the model
ensemble_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)]
# Model summary
ensemble_model.summary()


219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg19 (Functional)  │ (None, 7, 7, 512) │ 20,024,384 │ input_layer_14[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_resnet_v2 │ (None, 5, 5,      │ 54,336,736 │ input_layer_14[0… │
│ (Functional)        │ 1536)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_209 (Conv2D) │ (None, 7, 7, 256) │  1,179,904 │ vgg19[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_210 (Conv2D) │ (None, 5, 5, 256) │  3,539,200 │ inception_resnet… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 256) │      1,024 │ conv2d_209[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5, 5, 256) │      1,024 │ conv2d_210[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_10    │ (None, 3, 3, 256) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_11    │ (None, 2, 2, 256) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ max_pooling2d_10… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ max_pooling2d_11… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 512)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512)       │      2,048 │ concatenate_3[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 200)       │    102,600 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 200)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 6)         │      1,206 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 79,188,126 (302.08 MB)

 Trainable params: 59,101,150 (225.45 MB)

 Non-trainable params: 20,086,976 (76.63 MB)

In [ ]:
history = ensemble_model.fit(train_data,
                    epochs=10,
                    validation_data=validation_data,
                    callbacks=callbacks)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 349s 668ms/step - accuracy: 0.2220 - loss: 2.3138 - val_accuracy: 0.1914 - val_loss: nan
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 119s 601ms/step - accuracy: 0.2618 - loss: 1.7446 - val_accuracy: 0.2637 - val_loss: 1.6309
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 121s 611ms/step - accuracy: 0.2853 - loss: 1.6227 - val_accuracy: 0.2656 - val_loss: 2.2338
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 119s 599ms/step - accuracy: 0.3134 - loss: 1.5620 - val_accuracy: 0.2611 - val_loss: 1.6758
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 120s 606ms/step - accuracy: 0.3255 - loss: 1.5393 - val_accuracy: 0.2982 - val_loss: 2.1160
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 141s 601ms/step - accuracy: 0.3442 - loss: 1.5128 - val_accuracy: 0.2695 - val_loss: 2.9578
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 119s 602ms/step - accuracy: 0.3499 - loss: 1.4879 - val_accuracy: 0.2891 - val_loss: 1.8804
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 119s 601ms/step - accuracy: 0.3564 - lo

In [ ]:
predictions = ensemble_model.predict(data_test)

60/60 ━━━━━━━━━━━━━━━━━━━━ 31s 450ms/step


In [ ]:
stable_height = np.argmax(predictions, axis=1) + 1

output = pd.DataFrame({
    'id': test_data_df['id'],
    'stable_height': stable_height
})

output.to_csv('/content/drive/MyDrive/Colab Notebooks/predictions_2.csv', index=False)